In [1]:
%pip install pandoc python-docx openpyxl PyMuPDF transformers mammoth pywin32 Spire.Doc
%pip install doc2docx pytesseract
%pip install textract aspose-words


  Using cached pandoc-2.3.tar.gz (33 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement pywin32 (from versions: none)
ERROR: No matching distribution found for pywin32
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a c

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import os
import json
from docx import Document
import fitz  # PyMuPDF
from openpyxl import load_workbook
import aspose.words as aw
import spacy
import pytesseract
from PIL import Image

# Load a pre-trained NER model
nlp = spacy.load("en_core_web_sm")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/thinc/compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/thinc/compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


In [3]:
# List of questions to extract specific fields
questions = {
    "id": "What is the project ID?",
    "projectNo": "What is the project number?",
    "projectName": "What is the name of the project?",
    "projectDetail": "Can you provide a detailed description of the project?",
    "photoURL": "What is the photo URL associated with the project?", 
    "executingAgency": "Who is the executing agency for the project?",
    "status": "What is the status of the project? Is it completed, in progress, or not started?",
    "theme": "What are the main themes or focus areas of the project?",
    "estimatedCost": "What is the estimated cost of the project?",
    "budget": "What is the budget allocation for the project?",
    "totalDonatedAmount": "What is the total amount donated to the project?",
    "startDate": "What is the start date of the project?",
    "endDate": "What is the end date of the project?",
    # "Latitude": "What are the latitude coordinates of the project location?",
    # "Longitude": "What are the longitude coordinates of the project location?",
    "Locality_Name_EN": "What is the locality name in English?",
    "Locality_Name_AR": "What is the locality name in Arabic?",
    "Locality_PCODE": "What is the locality postal code?",
    "City_Name_EN": "What is the city name in English?",
    "City_Name_AR": "What is the city name in Arabic?",
    "City_PCODE": "What is the city postal code?",
    "District_Name_EN": "What is the district name in English?",
    "District_Name_AR": "What is the district name in Arabic?",
    "District_PCODE": "What is the district postal code?",
    "Municipal_Division_Type": "What is the municipal division type (kism, markaz, new city, or police-administered)?",
    "Municipal_Division_Name_EN": "What is the municipal division name in English?",
    "Municipal_Division_Name_AR": "What is the municipal division name in Arabic?",
    "Municipal_Division_PCODE": "What is the municipal division postal code?",
    "Governorate_Name_EN": "What is the governorate name in English?",
    "Governorate_Name_AR": "What is the governorate name in Arabic?",
    "Governorate_PCODE": "What is the governorate postal code?",
    "State_Name_EN": "What is the state name in English?",
    "State_Name_AR": "What is the state name in Arabic?",
    "State_PCODE": "What is the state postal code?",
    "Province_Name_EN": "What is the province name in English?",
    "Province_Name_AR": "What is the province name in Arabic?",
    "Province_PCODE": "What is the province postal code?",
    "Region_Name_EN": "What is the region name in English?",
    "Region_Name_AR": "What is the region name in Arabic?",
    "Region_PCODE": "What is the region postal code?",
    "Country_EN": "What is the country name in English?",
    "Country_AR": "What is the country name in Arabic?",
    "Country_PCODE": "What is the country postal code?",
    "donor": "Who are the donors for the project?",
    "contribution": "What contributions have been made to the project?",
    "dataReliability": "What is the data reliability rating for the project information?"
}

	1.	id: 1827
	2.	projectNo: PIMS 1237 CC MSP
	3.	projectName: Introduction of Vehicle Electric Bus Technology and Hybrid-Electric Bus Technology in Egypt – Phase 1a
	4.	projectDetail:
		•	The project aimed to introduce electric and hybrid-electric buses in Egypt to reduce pollution, improve public health, and protect national monuments from degradation due to heavy-duty buses.
		•	Phase 1a involved testing two electric buses, training local staff on maintenance and operations, and planning for subsequent phases.
	5.	photoURL: Not provided
	6.	executingAgency: [Egyptian Environmental Affairs Agency (EEAA), Social Fund for Development (SFD)]
	7.	status: Completed (Phase 1a)
	8.	theme: [Climate Change, Sustainable Transportation, Public Health, Cultural Heritage Protection]
	9.	estimatedCost: USD 1.714 million
	10.	budget:
		•	GEF: USD 0.7486 million
		•	EEAA: USD 0.3154 million
		•	SFD: USD 0.1 million
		•	In-kind contribution: USD 0.55 million
	11.	totalDonatedAmount: USD 0.7486 million
	12.	startDate: March 2000
	13.	endDate: June 2006 (Phase 1a)
	14.	Latitude: Not provided
	15.	Longitude: Not provided
	16.	Locality_Name_EN: Giza, Luxor
	17.	Locality_Name_AR: الجيزة، الأقصر
	18.	Locality_PCODE: Not provided
	19.	City_Name_EN: Cairo
	20.	City_Name_AR: القاهرة
	21.	City_PCODE: Not provided
	22.	District_Name_EN: Not provided
	23.	District_Name_AR: Not provided
	24.	District_PCODE: Not provided
	25.	Municipal_Division_Type: Not provided
	26.	Municipal_Division_Name_EN: Not provided
	27.	Municipal_Division_Name_AR: Not provided
	28.	Municipal_Division_PCODE: Not provided
	29.	Governorate_Name_EN: Giza Governorate, Luxor Governorate
	30.	Governorate_Name_AR: محافظة الجيزة، محافظة الأقصر
	31.	Governorate_PCODE: Not provided
	32.	State_Name_EN: Not applicable
	33.	State_Name_AR: Not applicable
	34.	State_PCODE: Not applicable
	35.	Province_Name_EN: Not applicable
	36.	Province_Name_AR: Not applicable
	37.	Province_PCODE: Not applicable
	38.	Region_Name_EN: Not applicable
	39.	Region_Name_AR: Not applicable
	40.	Region_PCODE: Not applicable
	41.	Country_EN: Egypt
	42.	Country_AR: مصر
	43.	Country_PCODE: Not provided
	44.	donor: [Global Environment Facility (GEF)]
	45.	contribution: USD 0.7486 million (GEF), USD 0.3154 million (EEAA), USD 0.1 million (SFD)
	46.	dataReliability: Information derived from final evaluation reports and project documents.

In [17]:
# Initialize the question-answering pipeline
qa_pipeline = pipeline("question-answering", model="NeuML/bert-small-cord19qa")

config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/140M [00:00<?, ?B/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at NeuML/bert-small-cord19qa were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (in

tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [18]:
# Function to extract information based on questions
def extract_info(text, questions):
    extracted_info = {}
    for key, question in questions.items():
        result = qa_pipeline(question=question, context=text)
        extracted_info[key] = result['answer']
    return extracted_info

In [19]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

In [20]:
# Function to extract text from XLSX
def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

In [21]:
# Function to convert DOC to DOCX and then extract text
def convert_doc_to_docx(doc_path):
    docx_path = doc_path + "x"
    doc = aw.Document(doc_path)
    doc.save(docx_path)
    return docx_path

In [22]:
# Function to extract text from DOC
def extract_text_from_doc(doc_path):
    docx_path = convert_doc_to_docx(doc_path)
    text = extract_text_from_docx(docx_path)
    
    os.remove(docx_path)
    return text

In [23]:
# Function to extract text from XLSX
def extract_data_from_xlsx(xlsx_path):
    wb = load_workbook(xlsx_path)
    ws = wb.active
    data = "\n".join(["\t".join(map(str, row)) for row in ws.iter_rows(values_only=True)])
    return data

In [24]:
# Main function to extract text based on file type
def extract_text_from_file(file_path):
    if file_path.endswith('.pdf'):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith('.docx'):
        return extract_text_from_docx(file_path)
    elif file_path.endswith('.doc'):
        return extract_text_from_doc(file_path)
    elif file_path.endswith('.xlsx'):
        return extract_data_from_xlsx(file_path)
    else:
        raise ValueError("Unsupported file format")

In [25]:
# Function to extract named entities from text using a specific model
def extract_entities(text):
    model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name)
    nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)
    entities = nlp(text)
    return entities

In [26]:
# Function to extract text from an image
def extract_text_from_image(image_path):
    return pytesseract.image_to_string(Image.open(image_path))

# Function to extract entities from text
def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

# Function to summarize text
def summarize_text(text, model="facebook/bart-large-cnn"):
    summarizer = pipeline("summarization", model=model)
    return summarizer(text, max_length=130, min_length=30, do_sample=False)

# Function to answer questions based on text
def answer_question(text, question, model="deepset/roberta-base-squad2"):
    qa_pipeline = pipeline("question-answering", model=model)
    return qa_pipeline(question=question, context=text)

In [27]:
# Function to map entities to a predefined schema
def map_entities_to_schema(entities):
    project_data = {
        "id": "",
        "projectNo": None,
        "projectName": "",
        "projectDetail": "",
        "photoURL": "",
        "executingAgency": [],
        "status": "",
        "theme": [],
        "estimatedCost": None,
        "budget": None,
        "totalDonatedAmount": None,
        "startDate": "",
        "endDate": "",
        "Latitude": None,
        "Longitude": None,
        "Locality_Name_EN": "",
        "Locality_Name_AR": "",
        "Locality_PCODE": "",
        "City_Name_EN": "",
        "City_Name_AR": "",
        "City_PCODE": "",
        "District_Name_EN": "",
        "District_Name_AR": "",
        "District_PCODE": "",
        "Municipal_Division_Type": "",
        "Municipal_Division_Name_EN": "",
        "Municipal_Division_Name_AR": "",
        "Municipal_Division_PCODE": "",
        "Governorate_Name_EN": "",
        "Governorate_Name_AR": "",
        "Governorate_PCODE": "",
        "State_Name_EN": "",
        "State_Name_AR": "",
        "State_PCODE": "",
        "Province_Name_EN": "",
        "Province_Name_AR": "",
        "Province_PCODE": "",
        "Region_Name_EN": "",
        "Region_Name_AR": "",
        "Region_PCODE": "",
        "Country_EN": "",
        "Country_AR": "",
        "Country_PCODE": "",
        "donor": [],
        "contribution": "",
        "dataReliability": ""
    }

    for entity in entities:
        if entity['entity_group'] == 'ID':
            project_data['id'] = entity['word']
        elif entity['entity_group'] == 'PROJECT_NO':
            project_data['projectNo'] = int(entity['word'])
        elif entity['entity_group'] == 'PROJECT_NAME':
            project_data['projectName'] += entity['word'] + " "
        elif entity['entity_group'] == 'PROJECT_DETAIL':
            project_data['projectDetail'] += entity['word'] + " "
        elif entity['entity_group'] == 'STATUS':
            project_data['status'] = entity['word']
        elif entity['entity_group'] == 'THEME':
            project_data['theme'].append(entity['word'])
        elif entity['entity_group'] == 'ESTIMATED_COST':
            project_data['estimatedCost'] = float(entity['word'].replace(',', ''))
        elif entity['entity_group'] == 'BUDGET':
            project_data['budget'] = float(entity['word'].replace(',', ''))
        elif entity['entity_group'] == 'TOTAL_DONATED_AMOUNT':
            project_data['totalDonatedAmount'] = float(entity['word'].replace(',', ''))
        elif entity['entity_group'] == 'START_DATE':
            project_data['startDate'] = entity['word']
        elif entity['entity_group'] == 'END_DATE':
            project_data['endDate'] = entity['word']
        elif entity['entity_group'] == 'LATITUDE':
            project_data['Latitude'] = float(entity['word'])
        elif entity['entity_group'] == 'LONGITUDE':
            project_data['Longitude'] = float(entity['word'])
        elif entity['entity_group'] == 'LOCALITY_NAME_EN':
            project_data['Locality_Name_EN'] = entity['word']
        elif entity['entity_group'] == 'LOCALITY_NAME_AR':
            project_data['Locality_Name_AR'] = entity['word']
        elif entity['entity_group'] == 'LOCALITY_PCODE':
            project_data['Locality_PCODE'] = entity['word']
        elif entity['entity_group'] == 'CITY_NAME_EN':
            project_data['City_Name_EN'] = entity['word']
        elif entity['entity_group'] == 'CITY_NAME_AR':
            project_data['City_Name_AR'] = entity['word']
        elif entity['entity_group'] == 'CITY_PCODE':
            project_data['City_PCODE'] = entity['word']
        elif entity['entity_group'] == 'DISTRICT_NAME_EN':
            project_data['District_Name_EN'] = entity['word']
        elif entity['entity_group'] == 'DISTRICT_NAME_AR':
            project_data['District_Name_AR'] = entity['word']
        elif entity['entity_group'] == 'DISTRICT_PCODE':
            project_data['District_PCODE'] = entity['word']
        elif entity['entity_group'] == 'MUNICIPAL_DIVISION_TYPE':
            project_data['Municipal_Division_Type'] = entity['word']
        elif entity['entity_group'] == 'MUNICIPAL_DIVISION_NAME_EN':
            project_data['Municipal_Division_Name_EN'] = entity['word']
        elif entity['entity_group'] == 'MUNICIPAL_DIVISION_NAME_AR':
            project_data['Municipal_Division_Name_AR'] = entity['word']
        elif entity['entity_group'] == 'MUNICIPAL_DIVISION_PCODE':
            project_data['Municipal_Division_PCODE'] = entity['word']
        elif entity['entity_group'] == 'GOVERNORATE_NAME_EN':
            project_data['Governorate_Name_EN'] = entity['word']
        elif entity['entity_group'] == 'GOVERNORATE_NAME_AR':
            project_data['Governorate_Name_AR'] = entity['word']
        elif entity['entity_group'] == 'GOVERNORATE_PCODE':
            project_data['Governorate_PCODE'] = entity['word']
        elif entity['entity_group'] == 'STATE_NAME_EN':
            project_data['State_Name_EN'] = entity['word']
        elif entity['entity_group'] == 'STATE_NAME_AR':
            project_data['State_Name_AR'] = entity['word']
        elif entity['entity_group'] == 'STATE_PCODE':
            project_data['State_PCODE'] = entity['word']
        elif entity['entity_group'] == 'PROVINCE_NAME_EN':
            project_data['Province_Name_EN'] = entity['word']
        elif entity['entity_group'] == 'PROVINCE_NAME_AR':
            project_data['Province_Name_AR'] = entity['word']
        elif entity['entity_group'] == 'PROVINCE_PCODE':
            project_data['Province_PCODE'] = entity['word']
        elif entity['entity_group'] == 'REGION_NAME_EN':
            project_data['Region_Name_EN'] = entity['word']
        elif entity['entity_group'] == 'REGION_NAME_AR':
            project_data['Region_Name_AR'] = entity['word']
        elif entity['entity_group'] == 'REGION_PCODE':
            project_data['Region_PCODE'] = entity['word']
        elif entity['entity_group'] == 'COUNTRY_EN':
            project_data['Country_EN'] = entity['word']
        elif entity['entity_group'] == 'COUNTRY_AR':
            project_data['Country_AR'] = entity['word']
        elif entity['entity_group'] == 'COUNTRY_PCODE':
            project_data['Country_PCODE'] = entity['word']
        elif entity['entity_group'] == 'DONOR':
            project_data['donor'].append(entity['word'])
        elif entity['entity_group'] == 'CONTRIBUTION':
            project_data['contribution'] = entity['word']
        elif entity['entity_group'] == 'DATA_RELIABILITY':
            project_data['dataReliability'] = entity['word']
    
    return project_data

In [28]:
# Function to extract information from files and save to JSON
def extract_info_from_files(file_paths, json_path):
    texts = [extract_text_from_file(file_path) for file_path in file_paths]
    combined_text = "\n".join(texts)
    
    # entities = extract_entities(combined_text)
    # Extract the information
    extracted_information = extract_info(combined_text, questions)
    # project_data = map_entities_to_schema(extracted_information)

    with open(json_path, 'w') as json_file:
        json.dump(extracted_information, json_file, indent=4)

In [29]:
file_paths = [
"/Users/izzymohamed/Downloads/Documents/PIMS 1237 CC MSP - ELECTRIC BUSES/Electric_buses_eval-Executive_Summary.doc",
"/Users/izzymohamed/Downloads/Documents/PIMS 1237 CC MSP - ELECTRIC BUSES/Electric_buses_Final_Evaluation_report.doc",
"/Users/izzymohamed/Downloads/Documents/PIMS 1237 CC MSP - ELECTRIC BUSES/TERMS_OF_REFERENCE.doc",
"/Users/izzymohamed/Downloads/Documents/PIMS 1237 CC MSP - ELECTRIC BUSES/report.xlsx"
]

json_path = "output.json"
extract_info_from_files(file_paths, json_path)